In [1]:
import pandas as pd
import numpy as np
import random
import os
import tensorflow as tf
import sys
sys.path.append('..')
from modules import utils, constants 
from modules.env import LupusEnv

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
SEED = 105
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

105

In [3]:
test_data = pd.read_csv('../new_data/test_set_constant.csv')
test_data = test_data.fillna(-1)
test_data.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,1,4,1,0,0,0,1,0,0,1


In [4]:
X_test = test_data.iloc[:, 0:-1]
y_test = test_data.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 24), (14000,))

In [5]:
model_name = 'dueling_dqn_per'
steps = int(100e6)

#### Delete from here

#### Pathways to test

In [6]:
# def get_distinct_pathways(test_df): #change to test_df after
#     unique_path_list = []
    
#     for path in test_df.trajectory.unique():
#         path_features = ast.literal_eval(path)
#         fts = [i for i in path_features if i not in ['Lupus', 'No lupus']]
#         print(path_features)
#         path_df = test_df[test_df.trajectory == path]
#         path_testing_df = testing_df.loc[path_df.index][fts]
#         path_ft_vals_vc = path_testing_df.value_counts()
#         path_unique_vals_lists = path_ft_vals_vc.index.to_list()
#         path_sample_nums = path_ft_vals_vc.values
        
#         for i in range(len(path_unique_vals_lists)):
#             unique_path_list.append(
#                 {'features':path_features, 'values':path_unique_vals_lists[i], 'sample_num':path_sample_nums[i]}
#             )
#     return unique_path_list

In [7]:
# def get_weighted_mean_with_path_values(test_df):
#     weighted_mean = 0
    
#     distinct_pathways = get_distinct_pathways(test_df)
#     #not yet implemented - seems it will always return 0 or 100 for the accuracy
#     return weighted_mean 

#### end here

In [8]:
# dqn_model_acc = utils.load_dqn(f'../models/logs/{model_name}/biopsy_3/seed_{SEED}_{steps}/best_acc_model.zip')
# test_df = utils.evaluate_dqn(dqn_model_acc, X_test, y_test)
# acc, f1, roc_auc = utils.test(test_df.y_actual, test_df.y_pred)
# acc_ft_score = utils.total_acc_feature_score(test_df)
# print(f'{model_name}: {acc_ft_score}, {acc}, {f1}, {roc_auc}, {test_df.episode_length.min()}, {test_df.episode_length.mean()}, {test_df.episode_length.max()}')
# print(f'{model_name}: {acc_ft_score}')

In [9]:
dqn_model_acc_ft = utils.load_dqn(f'../models/logs/{model_name}/missingness/0.3/biopsy_9/knn_imputer/default_mean_k_1/seed_{SEED}_{steps}/best_acc_model.zip')
# dqn_model_acc_ft = utils.load_dqn(f'../models/logs/{model_name}/biopsy_without_cost/seed_{SEED}_{steps}/best_acc_model.zip')
test_df = utils.evaluate_dqn(dqn_model_acc_ft, X_test, y_test)
acc, f1, roc_auc = utils.test(test_df.y_actual, test_df.y_pred)
pathway_score = utils.get_total_pathway_score(test_df)
pahm_score = utils.get_pahm_score(acc, pathway_score)
wpahm_score = utils.get_weighted_pahm_score([acc, pathway_score], [0.9, 0.1])
print(f'{model_name}: pathway score:{pathway_score}, pahm_score:{pahm_score}, wpahm_score:{wpahm_score}, acc:{acc}, f1:{f1}, roc_auc:{roc_auc}, {test_df.episode_length.min()}, {test_df.episode_length.mean()}, {test_df.episode_length.max()}')

Loading a model without an environment, this model cannot be trained until it has a valid environment.






Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







The environment seed is [126]
dueling_dqn_per: pathway score:22.967472915091893, pahm_score:37.01225109142022, wpahm_score:72.46021210712362, acc:95.27142857142857, f1:95.30518664897075, roc_auc:95.30859045590705, 5.0, 19.020357142857144, 25.0


In [10]:
# test_df.to_csv(f'../test_dfs/{model_name}_biopsy_3_best_path_pahm_{SEED}_{steps}.csv', index=False)

In [11]:
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,21.0,0.808826,1.0,1.0,"[fever, delirium, pericardial_effusion, low_c3...",0.0,1.0
1,1.0,24.0,0.681761,0.0,0.0,"[fever, delirium, pericardial_effusion, low_c3...",0.0,1.0
2,2.0,18.0,0.834015,1.0,1.0,"[fever, delirium, pericardial_effusion, low_c3...",0.0,1.0
3,3.0,19.0,0.834863,0.0,0.0,"[fever, delirium, pericardial_effusion, low_c3...",0.0,1.0
4,4.0,17.0,0.846448,1.0,1.0,"[fever, delirium, pericardial_effusion, low_c3...",0.0,1.0


In [12]:
test_df.y_pred.value_counts()

1.0    7114
0.0    6876
2.0      10
Name: y_pred, dtype: int64

### 